TMRM MFI Analysis, Mitochondrial Membrane Potential

Used to process experiments where the MFI of TMRM is measured
Static images
Z-stack images
Timecourse images

Input for all is a list of czi file names within a directory, and the path to that directory
All output a data frame with the image, grouping variable information, and MFI

In [5]:
# cell pose is machine learning model for segmenting cells
#from cellpose import models, utils, io

# for clearing jpynb output
from IPython.display import clear_output

# import necessary libraries
from scipy.ndimage import binary_fill_holes
from mpl_toolkits.mplot3d import Axes3D

import math
import skimage
import numpy as np
from czifile import CziFile
import czifile
from skimage import filters, morphology, segmentation
import numpy as np
from skimage.measure import regionprops
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.measure import label, regionprops
from skimage.io import imread
from skimage.measure import label
import czifile as czi
import pandas as pd
from skimage.filters import gaussian, threshold_otsu, sobel, median
from skimage.filters import threshold_otsu
from skimage import exposure
from skimage import morphology
from skimage.measure import label
from skimage.morphology import remove_small_objects
from skimage.morphology import binary_dilation, disk
from os import listdir
from os.path import isfile, join

Static Image Analysis

In [6]:
import czifile

def tmrm_mfi_single_image_analysis(file_name_list, folder_path):
    # define a function that applies a binary threshold to a single tmrm 
    def tmrm_mean_fluorescence_intensity(image_path):
        '''
        tmrm_mean_fluorescence_intensity(image_path) - applies a binary threshold to a single one-channel tmrm image.
        image_path: the filepath to the czi file to read
        Does not segment by cell
        Also prints the raw, adjusted, and masked image - only adds the image based on user input
        returns the mean fluorescent intensity of the mitochondria, determined by the segmentation
        '''
        # read in the czi file in as an array, and clean off the metadata
        image = czifile.imread(image_path)
        image_squeezed = np.squeeze(image)
        red_channel = image_squeezed[:]

        # adjust the contrast of the image via sigmoid
        contrast_adjusted_red = exposure.adjust_sigmoid(red_channel, cutoff=0.25)
        # normalize the contrast of the mitochondria
        contrast_adjusted_red_normalized = (contrast_adjusted_red - contrast_adjusted_red.min()) / (contrast_adjusted_red.max() - contrast_adjusted_red.min())    
        threshold_value_mitochondria = 0.12
        # create a binary mask
        mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria

        # plot the raw, contrast adjusted, and thresholded image
        fig, axes = plt.subplots(1, 3, figsize=(15, 9))
        # Plot each frame
        axes[0].imshow(red_channel)
        axes[0].set_title('red channel')
        axes[0].axis('off')

        axes[1].imshow(contrast_adjusted_red)
        axes[1].set_title('contrast adjusted image red channel')
        axes[1].axis('off')

        axes[2].imshow(mitochondria_thresholded)
        axes[2].set_title('threshold mitochondria')
        axes[2].axis('off')
        plt.show()

        # take the mean fluorescence where it is thresholded
        mean_intensity = np.mean(red_channel[mitochondria_thresholded > 0])
        return mean_intensity
    
    # initialize accumulators
    image_names = []
    mean_intensity_list = []
    # initialize list to store removed filenames in
    removed_files = []

    # iterate over each image in the folder and accumulate the image name and MFI
    for file in file_name_list:
        image_path = fr'{folder_path}\{file}'
        # use the function to find the MFI of the image
        image_mfi = tmrm_mean_fluorescence_intensity(image_path)

        # Jiya's chunk for visually inspecting each image as it is printed
        add_df = input("Do you want to add this dataframe? (yes/no): ").lower()
        plt.close('all')
        plt.clf()
        clear_output(wait=True)
        if 'y' in add_df:
         #if basename in result_dict and i in result_dict[basename]:
         # if segmetnation is good, add the results to be returned at the end
            image_names.append(file)
            mean_intensity_list.append(image_mfi)
        elif add_df == 'n':
            removed_files.append(file)
            #with open(r"C:\Users\bs1250\Box\LAB\Lab Folder\WGCNA_Ben\Analysis\discarded_images_06192024.txt", 'w') as f:
            #    f.write(f"Discarded: {file}")
            #f.close()
        elif add_df == 'stop':
            raise Exception('stop input')
           
    # format a data frame from the two lists from the two lists
    # does not write to an excel file, but it's output can
    mfi_df = pd.DataFrame(image_names)
    mfi_df.rename(columns={0: 'image'}, inplace=True)
    mfi_df.insert(1, "mean_intensity_red_channel", mean_intensity_list, True)

    # print the names of the removed files
    print('REMOVED FILES')
    for file in removed_files:
        print(file)

    return mfi_df

    #df.to_excel("mitochondrial_membrane_potential_MFI.xlsx")

In [ ]:
#test_path = r'C:\Users\bs1250\Box\LAB\Lab Folder\EXPERIMENTS\CONFOCAL EXPERIMENTS\Mitochondrial Membrane Potential\053024_CCCP_siRNA_MMMP_no_dox'

# test path for laptop
folder_to_analyze_static = "tmrm_mfi_static_images"

images_to_analyze = [f for f in listdir(folder_to_analyze_static) if isfile(join(folder_to_analyze_static, f))]
single_image_analysis  = tmrm_mfi_single_image_analysis(file_name_list=images_to_analyze, folder_path=folder_to_analyze_static)
display(single_image_analysis)

Z Stack Analysis

In [8]:
# define a file for taking the mean fluorescence intensity from z stacks
# will average and report the mean intensity of each slice within a stack - returning one value for each z stack

def tmrm_mfi_z_stack_analysis(file_name_list, folder_path):
    # define a function that applies a binary threshold to a single tmrm image
    def tmrm_mean_fluorescence_intensity_z_stack(image_path):
        '''
        tmrm_mean_fluorescence_intensity(image_path) - applies a binary threshold to a single one-channel tmrm image.
        image_path: the filepath to the czi file to read
        Does not segment by cell
        Also prints the raw, adjusted, and masked image - only adds the image based on user input
        returns the mean fluorescent intensity of the mitochondria, determined by the segmentation
        '''
        # read in the czi file in as an array, and clean off the metadata
        image = czifile.imread(image_path)
        image_squeezed = np.squeeze(image)

        # grab the number of slices
        n_slices = image_squeezed.shape[0]

        # initialize a list to accumulate MFIs from each slice
        slice_mean_intensity_list = []

        # iterate over each individual slice
        for current_slice in range(n_slices):
            unprocessed_slice = image_squeezed[current_slice, :, :] # take just the slice

            # grab the channel image from the larger czi file object
            red_slice = unprocessed_slice[:] # grab each pixel of the image array for this slice

            # adjust the contrast of the image via sigmoid
            contrast_adjusted_red = exposure.adjust_sigmoid(red_slice, cutoff=0.25)
            # normalize the contrast of the mitochondria
            #contrast_adjusted_red_normalized = (contrast_adjusted_red - contrast_adjusted_red.min()) / (contrast_adjusted_red.max() - contrast_adjusted_red.min())    
            #threshold_value_mitochondria = 0.08

            contrast_adjusted_red_normalized = (contrast_adjusted_red - contrast_adjusted_red.min()) / (contrast_adjusted_red.max() - contrast_adjusted_red.min())
            threshold_value_mitochondria = np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 1.85) #1.75 works well

            mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria

            # create a binary mask
            #mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria

            # plot the raw, contrast adjusted, and thresholded image
            fig, axes = plt.subplots(1, 3, figsize=(15, 9))
            # Plot each frame
            axes[0].imshow(red_slice)
            axes[0].set_title('red channel')
            axes[0].axis('off')

            axes[1].imshow(contrast_adjusted_red)
            axes[1].set_title('contrast adjusted image red channel')
            axes[1].axis('off')

            axes[2].imshow(mitochondria_thresholded)
            axes[2].set_title('threshold mitochondria')
            axes[2].axis('off')
            plt.show()

            # take the mean fluorescence where it is thresholded
            slice_mean_intensity = np.mean(red_slice[mitochondria_thresholded > 0])
            # add to the list for the z stacks
            slice_mean_intensity_list.append(slice_mean_intensity)


        # average the MFI from each slice and return the total MFI for the stack
        z_stack_mfi = np.mean(slice_mean_intensity_list)
        return z_stack_mfi
    
    # initialize accumulators
    image_names = []
    mean_intensity_list = []
    # initialize list to store removed filenames in
    removed_files = []    

    # iterate over each image in the folder and accumulate the image name and MFI
    for file in file_name_list:
        image_path = fr'{folder_path}\{file}'
        # use the function to find the MFI of the image
        image_mfi = tmrm_mean_fluorescence_intensity_z_stack(image_path)

        # Jiya's chunk for visually inspecting each image as it is printed
        add_df = input("Do you want to add this dataframe? (yes/no): ").lower()
        plt.close('all')
        plt.clf()
        clear_output(wait=True)
        if 'y' in add_df:
         #if basename in result_dict and i in result_dict[basename]:
         # if segmetnation is good, add the results to be returned at the end
            image_names.append(file)
            mean_intensity_list.append(image_mfi)
        elif add_df == 'n':
            removed_files.append(file)
            #with open(r"C:\Users\bs1250\Box\LAB\Lab Folder\WGCNA_Ben\Analysis\discarded_images_06192024.txt", 'w') as f:
            #    f.write(f"Discarded: {file}")
            #f.close()
        elif add_df == 'stop':
            raise Exception('stop input')
        
        #image_names.append(file)
        #mean_intensity_list.append(image_mfi)
   
    # format a data frame from the two lists from the two lists
    # does not write to an excel file, but it's output can
    mfi_df = pd.DataFrame(image_names)
    mfi_df.rename(columns={0: 'image'}, inplace=True)
    mfi_df.insert(1, "mean_intensity_red_channel", mean_intensity_list, True)

    # print the names of the removed files
    print('REMOVED FILES')
    for file in removed_files:
        print(file)

    return mfi_df
    
    

In [ ]:
folder_path_zstack = "tmrm_mfi_zstacks"

# take the file names of each file that is not an MIP in the file
images_to_analyze_zstack = [f for f in listdir(folder_path_zstack) if isfile(join(folder_path_zstack, f)) and 'MIP' not in f]

tmrm_mfi_zstack_results = tmrm_mfi_z_stack_analysis(file_name_list=images_to_analyze_zstack, folder_path=folder_path_zstack)

Timecourse Analysis

In [10]:
def tmrm_mfi_timecourse_analysis(file_name_list, folder_path):
    # initialize list of data frames to collect
    df_list = []
    # initialize lists to accumulate files that are removed after visual inspection
    removed_files = []
    def number_to_letter(number):
        if 1 <= number <= 26:
            return chr(number + ord('a') - 1)
        else:
            raise ValueError("Number out of range for conversion")

    # iterate through each image for analysis
    for path in file_name_list:
        image_path = fr'{folder_path}\{path}'

        # set the conditional to analyze the image as true
        analyze_timecourse = True

        try:
            # get path of image (this just ensures you are looking in the necessary folder since the images_to_analyze has just the file names)
            image = czifile.imread(image_path)
            image_squeezed = np.squeeze(image)
        except Exception:
            analyze_timecourse = False
            removed_files.append(path)
            image_squeezed = np.array([[0,1]])


        # iterate over the scene number (region)
        for image_slice in range(0,image_squeezed.shape[0]):
            # if a previous region has been rejected, skip analyzing the other ones
            if analyze_timecourse == False:
                print('got it region')
                continue

            # set accumulator lists for each slice (region/scene) - lists of lists to build into a mini df, which will be conc'ed
            slice_row_accum = []

            # iterate over the time number
            for time_stamp in range(0,image_squeezed.shape[1]):
                # skip analysis if you have already rejected an image from the timecourse
                if analyze_timecourse == False:
                    print('got it timestamps')
                    continue
                # print the time stamp and region for the images shown
                print("time stamp:", time_stamp+1)
                print("region:", number_to_letter(image_slice+1))

                red_channel = image_squeezed[image_slice,time_stamp,:,:]
                # these images, because there are multiple regions, show only as a small segment of a large image, so need to crop out the blank space

                # create a boolean array where there is any fluorescence
                image_over_0 = red_channel > 0
                # index each element where there is any fluorescence for the rows and columns, grab the max and min index
                rows_true = list(np.where(np.any(image_over_0, axis=1)))
                min_row = min(rows_true[0])
                max_row = max(rows_true[0])

                cols_true = np.where(np.any(image_over_0,axis=0))
                min_col = min(cols_true[0])
                max_col = max(cols_true[0])

                red_cropped = red_channel[min_row:max_row, min_col:max_col]
                
                # adjust the contrast of tmrm with sigmoid and then normalize it with the range of the image
                contrast_adjusted_red = exposure.adjust_sigmoid(red_cropped, cutoff=0.25)
                contrast_adjusted_red_normalized = (contrast_adjusted_red - contrast_adjusted_red.min()) / (contrast_adjusted_red.max() - contrast_adjusted_red.min())
                # manually set the threshold for the mitochondria segmentation and apply a binary mask
                threshold_value_mitochondria = 0.35
                mitochondria_thresholded = contrast_adjusted_red_normalized > threshold_value_mitochondria
                # calculate the MFI
                mean_intensity = np.mean(red_cropped[mitochondria_thresholded > 0])

                # accumulate by the slice - row of mini df
                slice_row_accum.append([path, mean_intensity, image_slice+1 , time_stamp+1])
            
                # plot the raw, contrast adjusted, and thresholded image
                fig, axes = plt.subplots(1, 3, figsize=(15, 9))
                # Plot the unprocessed red channel
                axes[0].imshow(red_cropped)
                axes[0].set_title('red channel')
                axes[0].axis('off')
                # print the contrast adjusted red
                axes[1].imshow(contrast_adjusted_red)
                axes[1].set_title('contrast adjusted image red channel')
                axes[1].axis('off')
                # print the binary mask
                axes[2].imshow(mitochondria_thresholded)
                axes[2].set_title('threshold mitochondria')
                axes[2].axis('off')
                plt.show()

            # generate a df from list of lists
            slice_df = pd.DataFrame(data=slice_row_accum, columns=['image', 'mean_intensity_red_channel', 'regions', 'timepoints'])

            # run this for each slice of the image

            # Jiya's chunk for visually inspecting each image as it is printed
            add_df = input("Do you want to add this region? (yes/no): ").lower()
            plt.close('all')
            plt.clf()
            clear_output(wait=True)
            if 'y' in add_df:
                # if segmentation is good, add df to list
                df_list.append(slice_df)
            elif 'n' in add_df:
                # if the region is not segmented well, add the file path to the list of removed files
                removed_files.append(path + " region " + number_to_letter(image_slice + 1))
                # set it to skip the other regions
                #with open(r"C:\Users\bs1250\Box\LAB\Lab Folder\WGCNA_Ben\Analysis\discarded_images_06192024.txt", 'w') as f:
                #    f.write(f"Discarded: {file}")
                #f.close()
            elif add_df == 'stop':
                raise Exception('stop input')
    
    # concatonate all collected data frames into one
    total_df = pd.concat(df_list)
    # replace region numbers with letters, reindex the data frame, and sort
    total_df['regions'] =  [number_to_letter(region) for region in total_df['regions']]
    total_df = total_df.reset_index(drop=True)
    total_df = total_df.sort_values(['image', 'regions', 'timepoints'], ascending=True)

    for i in removed_files:
        print(i)
        
    return total_df

    #df.to_excel("mitochondrial_membrane_potential_MFI.xlsx")

In [ ]:
filepath_timecourse = "tmrm_mfi_timecourse_images"
# extract all images/files from the folder of interest
images_to_analyze_timecourse = [f for f in listdir(filepath_timecourse) if isfile(join(filepath_timecourse, f))]

timecourse_results = tmrm_mfi_timecourse_analysis(images_to_analyze_timecourse, filepath_timecourse)
display(timecourse_results)
